ライブラリ

In [59]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

下準備

In [60]:
def split_message_to_bytes(message, num_blocks):
    # メッセージをスペースで分割して単語ごとのリストにする
    words = message.split()
    
    # 単語数をブロック数で分割する
    words_per_block = len(words) // num_blocks
    
    # 分割されたブロックを格納するリスト
    blocks = []

    # 単語数がブロック数で割り切れない場合、余りを処理するための変数
    remainder = len(words) % num_blocks
    
    # ブロックごとに単語を割り当てる
    start = 0
    for i in range(num_blocks):
        # 残りの余りが0でない場合、1つのブロックに余りを追加する
        if remainder > 0:
            block_length = words_per_block + 1
            remainder -= 1
        else:
            block_length = words_per_block
        
        # ブロックを作成し、ブロックリストに追加
        block = ' '.join(words[start:start+block_length])
        # ブロックをバイト表現に変換して追加
        block_bytes = block.encode('utf-8')
        blocks.append(block_bytes)
        
        # 次の開始位置を更新
        start += block_length

    return blocks

# テスト用メッセージ
m = "This message is for practicing AONT distributed processing."
# 分割するブロック数
num_blocks = 5

# メッセージを分割してブロックを取得
blocks_bytes = split_message_to_bytes(m, num_blocks)

# ブロックを出力
for i, block_bytes in enumerate(blocks_bytes):
    print(f"m_{i+1}: {block_bytes}")

m_1: b'This message'
m_2: b'is for'
m_3: b'practicing AONT'
m_4: b'distributed'
m_5: b'processing.'


In [61]:
# 鍵の長さ（バイト単位）
key_length = 16  # 例として16バイトの鍵を生成します

# ランダムなバイト列を生成して鍵として使用する
k = os.urandom(key_length)

# 共通鍵暗号の鍵を表示
print("共通鍵暗号の鍵:", k)

共通鍵暗号の鍵: b'g\xbc\xab\xfag\x0c\xb5\x84\xeb]~\xf3\xa7%\xe3\xa0'


In [62]:
# 鍵の長さ（バイト単位）
key_length = 16  # 例として16バイトの鍵を生成します

# ランダムなバイト列を生成して鍵として使用する
k_0 = os.urandom(key_length)

# 公開された共通鍵暗号の鍵を表示
print("公開された共通鍵暗号の鍵:", k_0)

公開された共通鍵暗号の鍵: b'4\x8d\x814.\xd5\x06\x9b-T\x1aR\x8e(\xc6\xa6'


In [63]:
def pad(data):
    # PKCS7 パディングを適用
    padding_length = 16 - (len(data) % 16)
    padded_data = data + bytes([padding_length] * padding_length)
    return padded_data

def Enc(key, plaintext):
    # 鍵と初期化ベクトル（IV）の生成
    iv = os.urandom(16)  # 16バイトのIVを生成
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # データをパディング
    padded_plaintext = pad(plaintext)

    # 暗号化
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    
    return iv + ciphertext

# 暗号化する鍵
key = b'\x00' * 16

# 暗号化するデータ
m = b'This message is for practicing AONT distributed processing.'

# 暗号化
encrypted_data = Enc(key, m)
print("任意の共通鍵暗号化:", encrypted_data)

任意の共通鍵暗号化: b'\xb0,\xc61\xaa\xbcY\x01=\x1f/\xb2\xfdf\xc7\x1f\xd7P\xe0\xff\xb2\xe6\xb5n\x90i}\x8b\x89\xdc\xae\xd38o\xe5@-\x1a\x80Y\x81\x8b\xf8\xd2\x86j\x8a\xec\xad\xf7\xa3\r\x94\x93\xcbk\x19\xbf\xd4\xbf\xdb\x99\xf4\x8bo\x87\xa8\xcdQ\xafe\x13c\xa2bV\xf3\xffK)'


＊　AONTによる秘密分散化の仕組み

１　s番目までの全てのブロックをシェアに変換する

In [64]:
# 各ブロックを m2_i に変換
m2_blocks = []
for i, block in enumerate(blocks, start=1):
    # ブロックをバイト列に変換
    block_bytes = block.encode('utf-8')
    
    # Enc 関数を使用して暗号化し、ブロックを変換
    encrypted_block = Enc(k, bytes([i]))  # i をバイト列に変換して暗号化に渡す
    m2_block = bytes(a ^ b for a, b in zip(block_bytes, encrypted_block))
    
    m2_blocks.append(m2_block)

# 結果を出力
for i, m2_block in enumerate(m2_blocks, start=1):
    print(f"m2_{i}: {m2_block}")

m2_1: b'\x8f/M\xc1\x1b\xcc\xf9\xab\xb7S\xd8\xf7'
m2_2: b' \x8a\xfa\xca.\xac'
m2_3: b'\x04\xd6h\x9a?\x91\x89\xbd\xf3\xa8\x92\x08\xc5\xf7>'
m2_4: b'u\xddk\xea\x7fHt\xea\x16<\xd0'
m2_5: b'\xe2\x9f#\xd3\xa9K\xc8\x91\x82Wb'


２　シェアのハッシュ値を得る

In [65]:
hash_values = []
for i, block in enumerate(blocks, start=1):
# m2_i + i の排他的論理和を計算
    xor_result = bytes(a ^ b for a, b in zip(block_bytes, bytes([i])))
    
    # Enc 関数を使用して暗号化し、ハッシュ値を計算
    hash_value = Enc(k_0, xor_result)
    
    hash_values.append(hash_value)

# 結果を出力
for i, hash_value in enumerate(hash_values, start=1):
    print(f"h_{i}: {hash_value}")


h_1: b'\x89\xf1\xfd\x87\x0c0\r6\xb3\xda\xa9L\x82\xfa\x19[\xc9\x90B\xad\xdd\xe5\x02\xc2\xf4\xa4\r\xc7\xb0._6'
h_2: b'\x97@\xe8J\x97X\x88q\x19?\x1e,E\x99\x86\xc2\xdfN\xfe\xf3\xbcw\xd9\x16\xa4\xb0^\x08\xc0;\n\xde'
h_3: b'\x8f\x83\x8f\x81/\xb6\x96?u\xf4\x12\\\x14\xd5\xc3\xe8\xf5H\x97\x8b\\\xc0\x17H\x8a|\xf3\xb9\xca\x033N'
h_4: b'\xb0\xf3\x15\x91\xc7_\x13\xfe]q\x0bTy\xe3i\xbc\x88\x0c\xcf\ru\x95J\xbdE\r"\x9a\x14\x80\x8bP'
h_5: b'\xa4\n\x1a\xf0M\xcc\x1e\xa4\x83h\xcc\x17\xf7.\xc0&\x06\n\x7f\xfc\x16\x1f\x9f\xcd@\xc2\xe9\xb3\xc2\xde<\xf9'


In [66]:
# m2_(s+1) を計算
m2_splus1 = k
for hash_value in hash_values:
    m2_splus1 = bytes(a ^ b for a, b in zip(m2_splus1, hash_value))

# 結果を出力
print(f"m2_(s+1): {m2_splus1}")

m2_(s+1): b'\xe2w>\xd7YA\xab\xa6\xeaU\x1c\x8c\xfa^\x16K'


４　共通鍵暗号の鍵 k を破棄する

In [67]:
# 共通鍵暗号の鍵 k を破棄する
del k

＊　復号の仕組み

１　シェアのハッシュ値を得る

In [68]:
for i, hash_value in enumerate(hash_values, start=1):
    print(f"h_{i}: {hash_value}")

h_1: b'\x89\xf1\xfd\x87\x0c0\r6\xb3\xda\xa9L\x82\xfa\x19[\xc9\x90B\xad\xdd\xe5\x02\xc2\xf4\xa4\r\xc7\xb0._6'
h_2: b'\x97@\xe8J\x97X\x88q\x19?\x1e,E\x99\x86\xc2\xdfN\xfe\xf3\xbcw\xd9\x16\xa4\xb0^\x08\xc0;\n\xde'
h_3: b'\x8f\x83\x8f\x81/\xb6\x96?u\xf4\x12\\\x14\xd5\xc3\xe8\xf5H\x97\x8b\\\xc0\x17H\x8a|\xf3\xb9\xca\x033N'
h_4: b'\xb0\xf3\x15\x91\xc7_\x13\xfe]q\x0bTy\xe3i\xbc\x88\x0c\xcf\ru\x95J\xbdE\r"\x9a\x14\x80\x8bP'
h_5: b'\xa4\n\x1a\xf0M\xcc\x1e\xa4\x83h\xcc\x17\xf7.\xc0&\x06\n\x7f\xfc\x16\x1f\x9f\xcd@\xc2\xe9\xb3\xc2\xde<\xf9'


２　共通鍵を復号する

In [69]:

# 排他的論理和を計算する関数
def xor_bytes(bytes_list):
    result = bytes_list[0]
    for b in bytes_list[1:]:
        result = bytes(a ^ b for a, b in zip(result, b))
    return result

# 式を計算
result = xor_bytes([m2_splus1] + hash_values)

print("結果:", result)

結果: b'g\xbc\xab\xfag\x0c\xb5\x84\xeb]~\xf3\xa7%\xe3\xa0'


３　シェアからデータmを復号する

In [72]:
# Enc 関数を使用して暗号化
encrypted_result = Enc(result, bytes([i]))

# m2_iと復号した共通鍵の排他的論理和を計算    
for i, m2_block in enumerate(m2_blocks, start=1):
    decrypted_block= bytes(a ^ b for a, b in zip(m2_block, encrypted_result))   
    print(f"復号されたm_{i}: ",decrypted_block)


復号されたm_1:  b'j\xc3\xf8\xb1\xa0seM(!\xfc]'
復号されたm_2:  b'\xc5fO\xba\x95\x13'
復号されたm_3:  b'\xe1:\xdd\xea\x84.\x15[l\xda\xb6\xa2|cy'
復号されたm_4:  b'\x901\xde\x9a\xc4\xf7\xe8\x0c\x89N\xf4'
復号されたm_5:  b'\x07s\x96\xa3\x12\xf4Tw\x1d%F'
